### Pong, Skizze

In [ ]:
import asyncio, threading
from ipycanvas import MultiCanvas
from pong_helpers import move_bar, move_ball, pts_on_circle

In [ ]:
# Angaben zu Ball und un Bar
N = 200
SIZE = 200
game = {'running': True,
        'fps': 2,
        'pts': pts_on_circle(N, 0.8 * SIZE/2, 0.9 * SIZE/2),
        'N': N,
        'bar': [50, 100, 50, 10],
        'bar_speed': 3,
        'bar_dir': 1, 
        'ball_idx': 0,
        'ball_speed': 1,
        'ball_dir': 1,
        'ball_r': 3,
        }

In [ ]:
# Tastensteuerung
def on_key_down(game, key, *flags):
    if key == 'ArrowRight':
        game['bar_dir'] = 1
    elif key == 'ArrowLeft':
        game['bar_dir'] = -1
    elif key == 'ArrowDown':  
        game['fps'] = max(1, game['fps'] - 5)
    elif key == 'ArrowUp':
        game['fps']  += 5
    elif key == ' ':
        game['running'] = False

In [ ]:
# Thread Variante
def start_loop_thread(game, bg, fg):  
    if game['running']:
        move_bar(game, bg)
        move_ball(game, fg)
        delay = 1/game['fps']
        t = threading.Timer(delay, start_loop_thread, args=[game, bg, fg])
        t.start()

In [ ]:
# async Variante
async def update_async(game, bg, fg):  
    while game['running']:
        move_bar(game, bg)
        move_ball(game, fg)
        delay = 1/game['fps']
        await asyncio.sleep(delay)
        
def start_loop_async(game, bg, fg):
    loop = asyncio.get_event_loop()
    task = loop.create_task(update_async(game, bg, fg))
    loop.call_soon_threadsafe(asyncio.ensure_future, task)
    return task

In [ ]:
# Canvas
mcanvas = MultiCanvas(2, width = 200, height = 200, 
                layout = {'border' : '1px solid black'},
               )
bg, fg = mcanvas
fg.fill_style = 'red'

mcanvas.on_key_down(lambda key,*flags:on_key_down(game, key, *flags))
# draw bar
bg.fill_rect(*game['bar'])
# draw_ball
pt = game['pts'][game['ball_idx']]
fg.fill_circle(*pt, 3)
mcanvas

In [ ]:
move_bar(game, bg)
move_ball(game, fg)

In [ ]:
# Async Variante
game['running'] = True
task = start_loop_async(game, bg, fg)
task

In [ ]:
# kann auch so loop stoppen
task.cancel()

In [ ]:
# Thread Variante
game['running'] = True
start_loop_tread(game, bg, fg)

In [ ]:
game['running'] = False